In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys 


liens_publications =[]

descriptions=[]

reactions = []

likes =[]
loves =[]
angryy =[]

produits=[]


def extract_publications_links():
    publications = driver.find_elements_by_tag_name('article')
    i=1
    for pub in publications:
        a = pub.find_element_by_xpath('/html/body/div/div/div[2]/div/div[1]/div[2]/div[2]/div[1]/div/section/article['+str(i)+']/footer/div[2]/a[1]')
        #print(a.get_attribute('href'))
        liens_publications.append(a.get_attribute('href'))
        i=i+1

    
    return(liens_publications)

def extract_description(lien):
    driver.get(lien)
    description = driver.find_element_by_xpath('/html/body/div/div/div[2]/div/div[1]/div[1]/div/div/div[1]/div/p')
    return(description.text)


def produit_par_publication(pub):
    res=""
    product_list=["double salire","credit","Crédithabitat","crédit" ,"9ardh","9arth","cartebancaire","DigiCarte","STBNet"]
    for p in product_list:
        if(type(pub)==list):
            break
        if(pub.find(p)!=-1):
            res=p
            break
        else:
            res="Neutre"
    return(res)

def nombre_reactions():
    i=1
    while(i<6):
        react = driver.find_element_by_xpath('/html/body/div/div/div[2]/div/div[1]/div[2]/div[2]/div[1]/div/section/article['+str(i)+']/footer/div[2]/span[1]/a[1]').text
        i=i+1
        #print(react)
        reactions.append(react)
    return(reactions)


driver = webdriver.Chrome('/Users/Dream/Desktop/Test/chromedriver')
driver.get('https://mbasic.facebook.com')


# Step 3) Search & Enter the Email or Phone field & Enter Password
username = driver.find_element_by_id("m_login_email")
password = driver.find_element_by_name("pass")
submit   = driver.find_element_by_name("login")
username.send_keys("benrejebfatma@yahoo.fr")
password.send_keys("bazinga22207009")
# Step 4) Click Login
submit.click()

confirm = driver.find_element_by_xpath('/html/body/div/div/div/div/table/tbody/tr/td/div/form/div/input')
confirm.click()

search = driver.find_element_by_xpath('/html/body/div/div/div[1]/div/form/table/tbody/tr/td[2]/input')
search.send_keys("SocieteTunisiennedeBanque")
search.send_keys(Keys.ENTER)


stb = driver.find_element_by_xpath('/html/body/div/div/div[2]/div[2]/div[1]/div[2]/div[1]/div[4]/div/div/section/div/div/div/div/article/div/header/table/tbody/tr/td[2]/header/h3/span/strong/a')
stb.click()

reactions = nombre_reactions()

extract_publications_links()

for lien in liens_publications:
    desc=extract_description(lien)
    descriptions.append(desc)

for d in descriptions:
    produit =  produit_par_publication(d)
    produits.append(produit)


    
for lien in liens_publications:
    driver.get(lien)
    react = driver.find_element_by_xpath('/html/body/div/div/div[2]/div/div[1]/div[2]/div/div[3]/a').get_attribute('href')
    driver.get(react)
    like = driver.find_element_by_xpath('/html/body/div/div/div[2]/div/table/tbody/tr/td/div/div/a[2]/span').text
    love = driver.find_element_by_xpath('/html/body/div/div/div[2]/div/table/tbody/tr/td/div/div/a[3]/span').text
    try:
        angry = driver.find_element_by_xpath('/html/body/div/div/div[2]/div/table/tbody/tr/td/div/div/a[4]/img').get_attribute('alt')
        if (angry=='Angry'):
            angry = driver.find_element_by_xpath('/html/body/div/div/div[2]/div/table/tbody/tr/td/div/div/a[4]/span').text
        else:
            angry='0'
    except:
        angry='0'
    likes.append(like)
    loves.append(love)
    angryy.append(angry)




In [7]:
import pandas as pd
# Create a Pandas dataframe from the data.
df = pd.DataFrame({'links': liens_publications,'description':descriptions, 'produit': produits ,'reactions': reactions ,'Like' : likes , 'love' : loves ,'Angry':angryy})

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('pandas_simple.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
writer.save()